*Notebook com o trabalho 01 da disciplina de Inferência Causal do Master em Jornalismo de Dados, Automação e Data Storytelling, disciplina ministrada por Lucas M. Novaes.*

**Grupo**: *Bianca Muniz, Jamile Santana, Jessica Avelar, Kaluan Bernardo e Thiago Araújo*

Os dados utilizados são da base do TSE e [podem ser acessados aqui](https://github.com/jessicaavelar/master-dados-automacao-data-storytelling-insper/tree/main/inferencia_causal/dados).

#### Objetivo

Utilizando os dados eleitorais, prepar os itens a seguir:
1. Uma tabela com três variáveis e as seguintes colunas: variável, média, mediana, desvio-padrão, valor mínimo (min), valor máximo (max);
2. Um histograma de uma variável contínua de alguma das bases;
3. Uma tabela de distribuição de alguma variável discreta;
4. Um gráfico de dispersão (scatterplot).

### Preparando a base de dados

In [1]:
import altair as alt
import pandas as pd

In [2]:
# Lendo os arquivos

prefeitos = pd.read_csv("prefeitos_2020_abridged.txt", sep=",", error_bad_lines=False)
resultados = pd.read_csv("results_2020_abridged.txt", sep=",", error_bad_lines=False)
despesas = pd.read_csv("despesas_2020_abridged.txt", sep=",",  error_bad_lines=False)

In [3]:
# Entendendo a base e os dados que estamos utilizando

prefeitos.head()

,ano_eleicao,num_turno,sigla_uf,sigla_ue,numero_candidato,cpf_candidato,nome_urna_candidato,sigla_partido,composicao_legenda,num_titulo_eleitoral_candidato,descricao_sexo,descricao_grau_instrucao,elected
1,2020,2,AC,1392,40,35942070210,SOCORRO NERI,PSB,PSB / PODE / PROS / SOLIDARIEDADE / PDT / PV /...,962152453,FEMININO,SUPERIOR COMPLETO,0
2,2020,1,AC,1112,11,5050940885,TIÃO FLORES,PP,PL / PP / MDB,453562402,MASCULINO,SUPERIOR COMPLETO,0
3,2020,1,AC,1473,65,13818805253,BATISTA,PC do B,PT / PC do B / PSB,3992496,MASCULINO,SUPERIOR COMPLETO,0
4,2020,1,AC,1074,15,72816082253,FAGNER SALES,MDB,PMN / MDB / PSDB / REPUBLICANOS / PSC / AVANTE...,3967302410,MASCULINO,SUPERIOR COMPLETO,0
5,2020,1,AC,1023,15,19666632268,ZÉ MARIA,MDB,MDB / PSDB / PROS / PL / SOLIDARIEDADE,657192402,MASCULINO,SUPERIOR COMPLETO,0


In [4]:
resultados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18127 entries, 1 to 18127
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ano_eleicao         18127 non-null  int64  
 1   num_turno           18127 non-null  int64  
 2   sigla_uf            18127 non-null  object 
 3   muni_code           18127 non-null  int64  
 4   sigla_partido_hoje  18127 non-null  object 
 5   total_votes_race    18127 non-null  int64  
 6   cand_votes          18127 non-null  int64  
 7   max_votes           18127 non-null  int64  
 8   runner_up           17943 non-null  float64
dtypes: float64(1), int64(6), object(2)
memory usage: 1.4+ MB


In [5]:
despesas.describe()

,ano,turno,muni_code,cnpj_campanha,numero_cand,cpf_cand,cpf_cnpj_fornecedor,cod_origem_despesa,valor
count,1440331.0,1.440331e+06,1.440331e+06,1.440331e+06,1.440331e+06,1.440331e+06,1.440331e+06,1.440331e+06,1.440331e+06
mean,2020.0,1.054621e+00,5.421150e+04,3.883218e+13,2.911549e+01,3.602378e+10,1.167734e+13,2.024354e+07,1.389667e+03
std,0.0,2.272400e-01,2.826184e+04,2.536918e+11,1.937870e+01,2.983496e+10,1.691860e+13,2.431867e+05,9.752909e+03
min,2020.0,1.000000e+00,1.900000e+01,3.830400e+13,1.000000e+01,1.661116e+06,-1.000000e+00,2.001000e+07,1.000000e-02
25%,2020.0,1.000000e+00,3.207700e+04,3.864600e+13,1.400000e+01,7.734571e+09,1.054721e+10,2.011000e+07,9.000000e+01
50%,2020.0,1.000000e+00,5.839400e+04,3.880730e+13,2.200000e+01,2.906330e+10,3.655799e+12,2.014000e+07,3.000000e+02
75%,2020.0,1.000000e+00,7.655400e+04,3.900366e+13,4.500000e+01,5.892708e+10,1.906729e+13,2.026000e+07,9.771050e+02
max,2020.0,2.000000e+00,9.907400e+04,4.302916e+13,9.000000e+01,9.999879e+10,9.859772e+13,2.090000e+07,2.500000e+06


In [6]:
# Para a análise, escolhemos trabalhar com o estado do RJ.
# Filtrando pelo estado:

prefeitos_rj = prefeitos.loc[prefeitos["sigla_uf"] == "RJ"]
resultados_rj = resultados.loc[resultados["sigla_uf"] == "RJ"]
despesas_rj = despesas.loc[despesas["uf"] == "RJ"]

In [7]:
# Renomear a coluna de identificação do candidato para o merge

despesas_rj = despesas_rj.rename(columns={"cpf_cand" : "cpf_candidato"})
despesas_rj

,ano,cargo,turno,uf,muni_code,muni,cnpj_campanha,numero_cand,nome_cand,cpf_candidato,partido,cpf_cnpj_fornecedor,nome_fornecedor,cod_origem_despesa,desc_geral,desc_detalhe,valor
895772,2020,Prefeito,1,RJ,58793,PIRAÍ,38864529000126,20,ARTHUR HENRIQUE GONÇALVES FERREIRA,9375155749,PSC,24613138000124,DG SIGN PUBLICIDADE GRAFICA E EDITORA LTDA,20110000,Publicidade por adesivos,VINIL PERF 0·80X0·40CM TUTUCA X ALFREDO MAGALHÃES,320.0
895773,2020,Prefeito,1,RJ,58793,PIRAÍ,38864529000126,20,ARTHUR HENRIQUE GONÇALVES FERREIRA,9375155749,PSC,24613138000124,DG SIGN PUBLICIDADE GRAFICA E EDITORA LTDA,20110000,Publicidade por adesivos,VINIL PERF 0·80X0·40CM TUTUCA X RITA BARROS,320.0
895774,2020,Prefeito,1,RJ,58793,PIRAÍ,38864529000126,20,ARTHUR HENRIQUE GONÇALVES FERREIRA,9375155749,PSC,24613138000124,DG SIGN PUBLICIDADE GRAFICA E EDITORA LTDA,20110000,Publicidade por adesivos,VINIL PERF 0·80X0·40CM TUTUCA X WILSON CAMARADA,320.0
895775,2020,Prefeito,1,RJ,58793,PIRAÍ,38864529000126,20,ARTHUR HENRIQUE GONÇALVES FERREIRA,9375155749,PSC,13553014750,ANDREA CRISTINA DE SOUZA,20800000,Atividades de militância e mobilização de rua,MILTANCIA DE RUA DE 08/10 A 15/11/2020,1430.0
895776,2020,Prefeito,1,RJ,58793,PIRAÍ,38864529000126,20,ARTHUR HENRIQUE GONÇALVES FERREIRA,9375155749,PSC,24613138000124,DG SIGN PUBLICIDADE GRAFICA E EDITORA LTDA,20140000,Publicidade por materiais impressos,SANTÃO 12 X 18CM TUTUCA X LAISA SOUZA,182.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999687,2020,Prefeito,1,RJ,58467,MESQUITA,38802829000180,55,CRISTIANE PELINCA DO AMARAL,2333247711,PSD,-1,#NULO#,20210000,"Encargos financeiros, taxas bancárias e/ou op....",TARIFA DE FOLHA DE CHEQUE,90.0
999688,2020,Prefeito,1,RJ,58971,SÃO GONÇALO,38563496000184,10,DEJORGE PATRICIO DA SILVA,3726063773,REPUBLICANOS,10678145741,ELAINE CRISTINA DIAS MIRANDA,20010000,Despesas com pessoal,ATIVIDADE POLITICA EM GERAL,300.0
999689,2020,Prefeito,1,RJ,58971,SÃO GONÇALO,38563496000184,10,DEJORGE PATRICIO DA SILVA,3726063773,REPUBLICANOS,21030397775,SARAH CONSTANTINO SOARES,20010000,Despesas com pessoal,ATIVIDADE POLITICA EM GERAL,300.0
999690,2020,Prefeito,1,RJ,58289,SÃO FRANCISCO DE ITABAPOANA,38955762000114,10,ANTÔNIO MARCOS PAPINHA DA SILVA,1759398705,REPUBLICANOS,8859130786,DANYELL BRAGA DIAS,20360001,Serviços advocatícios,PRESTAÇÃO DE SERVIÇOS JURÍDICOS,300.0


#### Exercício 1

Uma tabela com três variáveis e as seguintes colunas: variável, média, mediana, desvio-padrão, valor mínimo (min) e valor máximo (max).

In [8]:
# Total gasto por cada candidato a prefeito no RJ

despesas_rj["gasto_candi"] = despesas_rj.groupby(["cpf_candidato", "partido", "nome_cand"])["valor"].transform("sum")
despesas_candi = (despesas_rj[["nome_cand", "partido", "gasto_candi"]]).drop_duplicates().sort_values("gasto_candi",
                                                                                                      ascending=False)
# Cada candidato aparece mais de uma vez, "drop.cuplicates()" agrupa, transformando em um valor único,
# utilizamos também o "ascending" para colocar em ordem descrescente os gastos

despesas_candi

,nome_cand,partido,gasto_candi
896236,EDUARDO DA COSTA PAES,DEM,9368240.68
896171,MARTHA MESQUITA DA ROCHA,PDT,5195497.49
895973,MARCELO BEZERRA CRIVELLA,REPUBLICANOS,4867753.53
895897,BENEDITA SOUZA DA SILVA,PT,4865147.36
896187,LUIZ EDUARDO CARNEIRO DA SILVA DE SOUZA LIMA,PSL,3816528.51
...,...,...,...
906845,RUBENS DA COSTA LOPES FILHO,REDE,230.00
897164,LEANDRO FRANCISCO NEVES,AVANTE,209.50
949570,HUGO JOSE MORENO PEREIRA,PSTU,200.00
964510,LEONARDO STRAMARO PINTO,PRTB,150.00


In [9]:
# Análise descritiva do gasto por candidato

despesas_candi.describe()

,gasto_candi
count,5.470000e+02
mean,2.334723e+05
std,6.661096e+05
min,4.000000e+01
25%,2.269201e+04
50%,7.000000e+04
75%,1.854111e+05
max,9.368241e+06


In [10]:
# Total gasto por município

despesas_rj["gasto_cid"] = despesas_rj.groupby(["muni_code", "muni"])["valor"].transform("sum")
despesas_cid = (despesas_rj[["muni_code", "muni", "gasto_cid"]]).drop_duplicates().sort_values("gasto_cid", ascending=False)
print(despesas_cid)

        muni_code                   muni    gasto_cid
895897      60011         RIO DE JANEIRO  38089120.38
895887      58971            SÃO GONÇALO   8465982.86
895801      58190  CAMPOS DOS GOYTACAZES   5486725.44
895893      58653                NITERÓI   5478280.93
895842      58696            NOVA IGUAÇU   4953664.03
...           ...                    ...          ...
895797      58181                APERIBÉ    100406.85
910116      58319            DUAS BARRAS     98740.57
895791      58459         LAJE DO MURIAÉ     91945.00
898822      59056  SÃO SEBASTIÃO DO ALTO     91564.09
900459      58386                 MACUCO     65188.07

[92 rows x 3 columns]


In [11]:
# Total gasto por partido no RJ

despesas_rj["gasto_part"] = despesas_rj.groupby(["partido"])["valor"].transform("sum")
despesas_part = (despesas_rj)[["partido", "gasto_part"]].drop_duplicates().sort_values("gasto_part", ascending=False)

print(despesas_part)

              partido   gasto_part
895794            PDT  16401251.74
895798            DEM  15317213.41
895839             PT  11326638.61
895858   REPUBLICANOS  11186438.80
895836            PSL   9114358.38
895790             PL   8547897.03
895772            PSC   7789612.96
895791            MDB   6601476.49
895891             PP   6052869.84
895797            PSD   5258157.62
895838  SOLIDARIEDADE   4040082.97
895800      CIDADANIA   3163409.81
896066           PROS   2827586.58
896013           PSOL   2725342.52
895826         AVANTE   2683165.08
895869           PSDB   2667914.40
895917           REDE   2189077.86
896022            PTB   1851207.72
895845            PSB   1679172.66
898282           NOVO   1185046.64
895880            PTC    931033.28
896082           PODE    928072.43
895828            PMB    739541.93
895942           PRTB    651354.22
895851        PC do B    638148.60
897013       PATRIOTA    501086.74
896184             PV    294998.09
896158             D

In [12]:
despesas_part.describe()

,gasto_part
count,3.100000e+01
mean,4.119656e+06
std,4.574015e+06
min,1.600000e+03
25%,6.954481e+05
50%,2.667914e+06
75%,6.327173e+06
max,1.640125e+07


#### Exercício 2

Um histograma de uma variável contínua de alguma das bases.

In [13]:
# Histograma com o gasto dos candidatos.

alt.Chart(despesas_candi).mark_bar().encode(
    alt.X("gasto_candi:Q", title = "Total gasto", bin=True),
    alt.Y("count()", title = "Candidatos"),
    tooltip = ["count()"]
).properties(title= "Gastos dos candidatos RJ")

alt.Chart(...)

In [14]:
# Histograma com o gasto dos municípios.

alt.Chart(despesas_cid).mark_bar().encode(
    alt.X("gasto_cid:Q", title = "Total gasto", bin=True),
    alt.Y("count()", title = "Cidades"),
    tooltip = ["count()"]
).properties(title= "Gastos dos municípios RJ")

alt.Chart(...)

In [15]:
# Histograma com o gasto dos partidos no RJ.

alt.Chart(despesas_part).mark_bar().encode(
    alt.X("gasto_part:Q", title = "Total gasto", bin=True),
    alt.Y("count()", title = "Partidos"),
    tooltip = ["count()"]
).properties(title= "Gastos dos partidos no RJ")

alt.Chart(...)

#### Exercício 3

Uma tabela de distribuição de alguma variável discreta.

In [16]:
# Gasto total por categoria de despesas

gastos = despesas_rj[["desc_geral", "valor"]]
gastos = (gastos.groupby(["desc_geral"]).sum().sort_values("valor", ascending=False))
gastos

,valor
desc_geral,
Publicidade por materiais impressos,22406289.10
Serviços prestados por terceiros,19093156.94
Despesas com pessoal,14432652.48
"Produção de programas de rádio, televisão ou vídeo",14100811.99
Publicidade por adesivos,12286469.39
Serviços advocatícios,7063638.59
Doações financeiras a outros candidatos/partidos,6201420.36
Atividades de militância e mobilização de rua,5494122.96
Despesa com Impulsionamento de Conteúdos,5194626.90


#### Exercício 4

Um gráfico de dispersão.

In [17]:
# "candidatos" nos dá a quantidade de candidatos por partido no RJ

despesas_rj["candidatos"] = despesas_rj.groupby(["partido"])["nome_cand"].transform("nunique")


# gastos_partido nos dá a quantidade que foi gasta por partido e o número de candidatos por partido

gastos_partido = despesas_rj[["partido", "gasto_part", "candidatos"]]
gastos_partido = gastos_partido.drop_duplicates()
gastos_partido = gastos_partido.sort_values(by = "gasto_part", ascending = False)
gastos_partido

,partido,gasto_part,candidatos
895794,PDT,16401251.74,29
895798,DEM,15317213.41,27
895839,PT,11326638.61,24
895858,REPUBLICANOS,11186438.80,34
895836,PSL,9114358.38,21
895790,PL,8547897.03,27
895772,PSC,7789612.96,34
895791,MDB,6601476.49,25
895891,PP,6052869.84,24
895797,PSD,5258157.62,31


In [18]:
alt.Chart(gastos_partido).mark_point().encode(
    alt.X("candidatos", title = "Número de candidatos"),
    alt.Y("gasto_part", title = "Total gasto do partido"),
    tooltip = ["partido", "candidatos", "gasto_part"]
).properties(title = "Relação do número de candidatos e total gasto por partido")

alt.Chart(...)

In [19]:
# Média de gastos dos partidos por candidato

gastos_partido["media_candi"] = gastos_partido["gasto_part"] / gastos_partido["candidatos"]


alt.Chart(gastos_partido).mark_point().encode(
    alt.X("candidatos", title = "Número de candidatos"),
    alt.Y("media_candi", title = "Média de gasto por candidato"),
    tooltip = ["partido", "candidatos", "gasto_part"]
).properties(title = "Média de gastos dos partidos por candidato")

alt.Chart(...)